Carga de datos y verificación de archivos de datos.

In [ ]:
import pyarrow.parquet as pq
import pandas as pd

# Especifica la ruta del archivo Parquet
ruta_archivo = "/content/drive/MyDrive/No Country Proyecto/Data/Clean/Icfes2023_feature_engineering.parquet"

# ver si el archivo es un archivo Parquet
try:
    pq.ParquetFile(ruta_archivo)
    print("El archivo es un archivo Parquet válido.")
except Exception as e:
    print("Error:", e)

El archivo es un archivo Parquet válido.


In [ ]:
# Lee el archivo Parquet en un DataFrame de pandas
df = pd.read_parquet(ruta_archivo)

In [1]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [ ]:
print(df.head())

   ESTU_TIPODOCUMENTO ESTU_GENERO  PERIODO ESTU_DEPTO_RESIDE  \
4                  TI           F        1             VALLE   
6                  TI           F        1             VALLE   
7                  TI           M        1             VALLE   
10                 TI           F        1             VALLE   
13                 TI           M        1            NARIÑO   

   ESTU_MCPIO_RESIDE ESTU_PRESENTACIONSABADO  FAMI_ESTRATOVIVIENDA  \
4               CALI                      No                     2   
6         CAICEDONIA                      No                     1   
7            JAMUNDÍ                      No                     1   
10              CALI                      No                     3   
13             PASTO                      No                     4   

   FAMI_PERSONASHOGAR  FAMI_CUARTOSHOGAR  \
4               1 a 2                2.0   
6               3 a 4                2.0   
7               5 a 6                2.0   
10              5 

In [ ]:
# Imprimir la lista de columnas y tipos de datos
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 180413 entries, 4 to 551147
Data columns (total 37 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ESTU_TIPODOCUMENTO           180413 non-null  object 
 1   ESTU_GENERO                  180413 non-null  object 
 2   PERIODO                      180413 non-null  int64  
 3   ESTU_DEPTO_RESIDE            180413 non-null  object 
 4   ESTU_MCPIO_RESIDE            180413 non-null  object 
 5   ESTU_PRESENTACIONSABADO      180413 non-null  object 
 6   FAMI_ESTRATOVIVIENDA         180413 non-null  int64  
 7   FAMI_PERSONASHOGAR           180413 non-null  object 
 8   FAMI_CUARTOSHOGAR            180413 non-null  float64
 9   FAMI_EDUCACIONPADRE          180413 non-null  object 
 10  FAMI_EDUCACIONMADRE          180413 non-null  object 
 11  FAMI_TRABAJOLABORPADRE       180413 non-null  int64  
 12  FAMI_TRABAJOLABORMADRE       180413 non-null  int64  
 13  COLE

voy a usar el random state 42 en todo el notebook para poder comparar con otros modelos el mismo resultado.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


# Separar la variable objetivo 'PUNT_GLOBAL' del resto de características
X = df.drop(columns=['PUNT_GLOBAL'])  # Características
y = df['PUNT_GLOBAL']  # Variable objetivo

# Codificar las variables categóricas en X
X_encoded = pd.get_dummies(X)

# Codificar la variable objetivo si es categórica
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_encoded, X_test_encoded, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_encoded, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test_encoded)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Error Cuadrático Medio:", mse)



Error Cuadrático Medio: 4.258720602499793


LABEL ENCODER MODELO GUARDADO.

In [ ]:
import pickle

# Guardar el objeto LabelEncoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Puedes cargar el LabelEncoder en cualquier momento usando:
# with open('label_encoder.pkl', 'rb') as f:
#     loaded_label_encoder = pickle.load(f)


MODELO RF GUARDADO EN DRIVE USANDO PICKLE.

In [ ]:
import pickle

# Guardar el modelo entrenado
with open('random_forest_model_new.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

# Cargar el modelo en cualquier momento usando:
# with open('random_forest_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)


In [ ]:
import joblib

# Guardar el modelo entrenado
joblib.dump(rf_model, 'random_forest_model_icfe.pkl')

# Ahora puedes cargar el modelo en cualquier momento usando:
# loaded_model = joblib.load('random_forest_model.pkl')


['random_forest_model_icfe.pkl']

In [ ]:
#/content/random_forest_model.pkl

In [ ]:
# with open('random_forest_model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

Random Forest error cuadratico medio 4.259, 20/25 minutos aprox

Regresion lineal.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Inicializar OneHotEncoder sin categorías conocidas y con handle_unknown='ignore'
encoder = OneHotEncoder(handle_unknown='ignore')

# Crear un objeto ColumnTransformer para aplicar la codificación a las columnas categóricas
column_transformer = ColumnTransformer([('encoder', encoder, categorical_cols)], remainder='passthrough')

# Aplicar la transformación a X_train y X_test
X_train_encoded = column_transformer.fit_transform(X_train)
X_test_encoded = column_transformer.transform(X_test)

# Inicializar y entrenar el modelo de regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train_encoded, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_linear = linear_model.predict(X_test_encoded)

# Calcular el MSE
mse_linear = mean_squared_error(y_test, y_pred_linear)
print("MSE Regresión Lineal:", mse_linear)


MSE Regresión Lineal: 34.101160194690934


MSE lineal: 34.101

Regresión Polinomica

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Inicializar OneHotEncoder sin categorías conocidas y con handle_unknown='ignore'
# esto es porque hay muchas variables categóricas dentro de cada variable categorica
encoder = OneHotEncoder(handle_unknown='ignore')

# Crear un objeto ColumnTransformer para aplicar la codificación a las columnas categóricas
column_transformer = ColumnTransformer([('encoder', encoder, categorical_cols)], remainder='passthrough')

# Aplicar la transformación a X_train y X_test
X_train_encoded = column_transformer.fit_transform(X_train)
X_test_encoded = column_transformer.transform(X_test)


# Inicializar y entrenar el modelo de regresión polinómica
degree = 2  # Grado del polinomio
poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
poly_model.fit(X_train_encoded, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_poly = poly_model.predict(X_test_encoded)

# Calcular el MSE
mse_poly = mean_squared_error(y_test, y_pred_poly)
print("MSE Regresión Polinómica (Grado {}):".format(degree), mse_poly)




MSE Regresión Polinómica (Grado 2): 11.849037965538459


Regresión lineal y polinomica usando solo variables numéricas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

# Seleccionar solo las columnas numéricas
numeric_columns = df.select_dtypes(include=['int64', 'float64'])

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(numeric_columns.drop('PUNT_GLOBAL', axis=1), numeric_columns['PUNT_GLOBAL'], test_size=0.2, random_state=42)

# Regresión lineal
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Evaluar el modelo lineal
linear_predictions = linear_model.predict(X_test)
linear_mse = mean_squared_error(y_test, linear_predictions)
print("Error cuadrático medio (regresión lineal):", linear_mse)

# Regresión polinomial
polynomial_features = PolynomialFeatures(degree=2)
X_poly = polynomial_features.fit_transform(X_train)
polynomial_model = LinearRegression()
polynomial_model.fit(X_poly, y_train)

# Evaluar el modelo polinomial
X_test_poly = polynomial_features.transform(X_test)
poly_predictions = polynomial_model.predict(X_test_poly)
poly_mse = mean_squared_error(y_test, poly_predictions)
print("Error cuadrático medio (regresión polinomial):", poly_mse)


Error cuadrático medio (regresión lineal): 59.74432581654054
Error cuadrático medio (regresión polinomial): 32.601482615225805


Red Neural
(solo variables numéricas)




Este código utiliza una red neuronal básica con dos capas ocultas. La primera capa tiene 64 neuronas y la segunda tiene 32. La función de activación utilizada es ReLU en las capas ocultas. La capa de salida no tiene función de activación ya que estamos tratando de predecir un valor numérico.

Después de compilar el modelo, lo entrenamos durante 50 épocas utilizando el conjunto de entrenamiento. Luego, evaluamos el modelo utilizando el conjunto de prueba y calculamos el error cuadrático medio.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


# Seleccionar solo las columnas numéricas y la variable objetivo
numeric_columns = df.select_dtypes(include=['int64', 'float64'])
X = numeric_columns.drop('PUNT_GLOBAL', axis=1).values
y = numeric_columns['PUNT_GLOBAL'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definir el modelo de red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenar el modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluar el modelo
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio (red neuronal):", mse)


Epoch 1/50
3609/3609 [==============================] - 10s 2ms/step - loss: 3774.1670 - val_loss: 53.8669
Epoch 2/50
3609/3609 [==============================] - 9s 3ms/step - loss: 33.4750 - val_loss: 21.4225
Epoch 3/50
3609/3609 [==============================] - 10s 3ms/step - loss: 18.5334 - val_loss: 17.0428
Epoch 4/50
3609/3609 [==============================] - 8s 2ms/step - loss: 15.5531 - val_loss: 19.2817
Epoch 5/50
3609/3609 [==============================] - 9s 3ms/step - loss: 14.0494 - val_loss: 13.6820
Epoch 6/50
3609/3609 [==============================] - 10s 3ms/step - loss: 13.1448 - val_loss: 14.0974
Epoch 7/50
3609/3609 [==============================] - 8s 2ms/step - loss: 12.7077 - val_loss: 12.2336
Epoch 8/50
3609/3609 [==============================] - 12s 3ms/step - loss: 12.3594 - val_loss: 11.9716
Epoch 9/50
3609/3609 [==============================] - 9s 3ms/step - loss: 11.9292 - val_loss: 11.7870
Epoch 10/50
3609/3609 [==============================] - 8

Red Neural con todas las variables, usando incrustaciones de texto (Embeddings)
Esto está incompleto, hay que limpiar, ordenar y rellenar los datos.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Dividir los datos en características (X) y variable objetivo (y)
X = df.drop('PUNT_GLOBAL', axis=1)
y = df['PUNT_GLOBAL']

# Dividir los datos en características categóricas y numéricas
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir variables categóricas en variables dummy (one-hot encoding)
X_train_categorical = pd.get_dummies(X_train[categorical_features])
X_test_categorical = pd.get_dummies(X_test[categorical_features])

# Normalizar las características numéricas
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_numeric_scaled = scaler.transform(X_test[numeric_features])


keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


# Combinar características numéricas y categóricas
X_train_processed = np.hstack((X_train_categorical.values, X_train_numeric_scaled))
X_test_processed = np.hstack((X_test_categorical.values, X_test_numeric_scaled))

# Definir la arquitectura del modelo
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=['accuracy'])

model.build(input_shape=X_train_processed.shape)
model.summary()







Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (144330, 300)             248100    
                                                                 
 dense_1 (Dense)             (144330, 100)             30100     
                                                                 
 dense_2 (Dense)             (144330, 10)              1010      
                                                                 
Total params: 279210 (1.07 MB)
Trainable params: 279210 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
